In [1]:
%%configure -f 
{
    "conf":
    {
        "spark.app.name":"Bhavesh_Inventory_prediction",
        "spark.yarn.queue": "workflow",
        "spark.jars": "/apps/Jars/obelisk-retail-legos.jar,/apps/Jars/mysql-connector-java-5.1.40.jar",
        "spark.driver.extraJavaOptions": "-Dscala.color",
        "spark.driver.memory": "4g",
        "spark.executor.memory": "4g",
        "spark.executor.instances": "4"
    }
}

In [2]:
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.Constants.STANDARD_COL_NAMES._
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.io._
import ai.couture.obelisk.commons.utils.DateTimeUtil._
import org.apache.spark.sql._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import spark.implicits._

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1388,application_1685341929318_2162,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.Constants.STANDARD_COL_NAMES._
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.io._
import ai.couture.obelisk.commons.utils.DateTimeUtil._
import org.apache.spark.sql._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import spark.implicits._


In [3]:
import java.text.SimpleDateFormat
import java.time.DayOfWeek.{MONDAY, SUNDAY}
import java.time.format.DateTimeFormatter
import java.time.format.DateTimeFormatter.ofPattern
import java.time.temporal.TemporalAdjusters.previousOrSame
import java.time.temporal.{ChronoUnit, WeekFields}
import java.time.{LocalDate, LocalDateTime}
import java.util.{Date, Locale, TimeZone}  
def getFutureMonthDateFromHere(numMonths: Int, date: String, inputDateFormat: String,
                                 outputDateFormat: String = "yyyy-MM-dd"): String = {
    LocalDate.parse(date, ofPattern(inputDateFormat)).plusMonths(numMonths).format(ofPattern(outputDateFormat))
  }

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.text.SimpleDateFormat
import java.time.DayOfWeek.{MONDAY, SUNDAY}
import java.time.format.DateTimeFormatter
import java.time.format.DateTimeFormatter.ofPattern
import java.time.temporal.TemporalAdjusters.previousOrSame
import java.time.temporal.{ChronoUnit, WeekFields}
import java.time.{LocalDate, LocalDateTime}
import java.util.{Date, Locale, TimeZone}
getFutureMonthDateFromHere: (numMonths: Int, date: String, inputDateFormat: String, outputDateFormat: String)String


In [3]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, SparkSession}
import java.time.format.DateTimeFormatter.ofPattern
import java.time.temporal.{ChronoUnit}
import java.time.{LocalDate}
import org.apache.spark.sql.types._

  val getNumOfMonthsBetweenTwoDates = udf((startDate: String, endDate: String)=> {

    val inputDateFormat: String = "yyyy-MM-dd"
    val dateBefore: LocalDate = LocalDate.parse(startDate, ofPattern(inputDateFormat))

    val dateAfter: LocalDate = LocalDate.parse(endDate, ofPattern(inputDateFormat))

    ChronoUnit.MONTHS.between(dateBefore, dateAfter)

  })

  val getOldMonthDateFromHere = udf((numMonths: Int, date: String) => {
      val inputDateFormat: String = "yyyy-MM-dd"
    val outputDateFormat: String = "yyyy-MM-dd"

    LocalDate.parse(date, ofPattern(inputDateFormat)).minusMonths(numMonths).format(ofPattern(outputDateFormat))
  })


    def getFutureMonthDateFromHere(numMonths: Int, date: String, inputDateFormat: String,
                                 outputDateFormat: String = "yyyy-MM-dd"): String = {
    LocalDate.parse(date, ofPattern(inputDateFormat)).plusMonths(numMonths).format(ofPattern(outputDateFormat))
  }


  def getFutureDateFromHere(numDays: Int, date: String, inputDateFormat: String,
                            outputDateFormat: String = "yyyy-MM-dd"): String = {
    LocalDate.parse(date, ofPattern(inputDateFormat)).plusDays(numDays).format(ofPattern(outputDateFormat))
  }

    def getOldDateFromHere(numDays: Int, date: String, inputDateFormat: String,
                         outputDateFormat: String = "yyyy-MM-dd"): String = {
    LocalDate.parse(date, ofPattern(inputDateFormat)).minusDays(numDays).format(ofPattern(outputDateFormat))
  }

  def getOldMonthDateFromHerenotUdf(numMonths: Int, date: String, inputDateFormat: String,
                              outputDateFormat: String = "yyyy-MM-dd"): String = {
    LocalDate.parse(date, ofPattern(inputDateFormat)).minusMonths(numMonths).format(ofPattern(outputDateFormat))
  }

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, SparkSession}
import java.time.format.DateTimeFormatter.ofPattern
import java.time.temporal.ChronoUnit
import java.time.LocalDate
import org.apache.spark.sql.types._
getNumOfMonthsBetweenTwoDates: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function2>,LongType,Some(List(StringType, StringType)))
getOldMonthDateFromHere: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function2>,StringType,Some(List(IntegerType, StringType)))
getFutureMonthDateFromHere: (numMonths: Int, date: String, inputDateFormat: String, outputDateFormat: String)String
getFutureDateFromHere: (numDays: Int, date: String, inputDateFormat: String, outputDateFormat: String)String
getOldDateFromHere: (numDays: Int, date: String, inputDateFormat: String, outputDateFormat: String)String
getOldMonthDateFromHerenotUdf: (numMonths: Int, date: String, inputDateFormat: String, outputDateFor

In [3]:
var DATE="date"
var PREDICTED="predicted"
var ACTUAL="actual"

var rundate = "2023-04-30"
var dateForWhichPredictionsMade = getFutureDateFromHere(1,rundate,"yyyy-MM-dd")
var predictionend = "2023-05-31"
var trainendDate =getOldDateFromHere(1,rundate,"yyyy-MM-dd")
var trainStartDate = "2022-02-01"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DATE: String = date
PREDICTED: String = predicted
ACTUAL: String = actual
rundate: String = 2023-04-30
dateForWhichPredictionsMade: String = 2023-05-01
predictionend: String = 2023-05-31
trainendDate: String = 2023-04-29
trainStartDate: String = 2022-02-01


In [4]:
def saveNLoadDF(key: String)(df: DataFrame): DataFrame = {
    var tempPath = "/data/Archive/bhavesh/InventoryPrediction/temp/dashboard_metrics"
    DFToParquet.putDF(tempPath + "/" + key, df)
    ParquetToDF.getDF(tempPath + "/" + key)
  }

def load(key: String): DataFrame = {
    var tempPath = "/data/Archive/bhavesh/InventoryPrediction/temp/dashboard_metrics"
    ParquetToDF.getDF(tempPath + "/" + key)
}

val getOldDateFromHereUDF = udf((numDays: Int, date: String, inputDateFormat: String, outputDateFormat: String) => {
    getOldDateFromHere(numDays, date, inputDateFormat, outputDateFormat)
})

def getMonths(dateFormat: String)(df: DataFrame): DataFrame = {
    df.withColumn("dateFormatted", getOldDateFromHereUDF(lit(0), col(DATE), lit(dateFormat), lit("yyyy-MM-dd")))
    .withColumn("year", split(col("dateFormatted"), "-")(0))
    .withColumn("month", split(col("dateFormatted"), "-")(1))
    .withColumn("monthsInHis", dense_rank().over(Window.orderBy(desc("year"), desc("month"))) - lit(1))
    .drop("dateFormatted", "year", "month")
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

saveNLoadDF: (key: String)(df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame
load: (key: String)org.apache.spark.sql.DataFrame
getOldDateFromHereUDF: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function4>,StringType,Some(List(IntegerType, StringType, StringType, StringType)))
getMonths: (dateFormat: String)(df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


In [5]:
var baseDir = "/data/Archive/bhavesh/InventoryPrediction"
var runDir = s"$baseDir/$rundate"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

baseDir: String = /data/Archive/bhavesh/InventoryPrediction
runDir: String = /data/Archive/bhavesh/InventoryPrediction/2023-04-30


In [6]:
var productLevelDailyQuantity = ParquetToDF.getDF(s"$runDir/DailyQuantity/product")
productLevelDailyQuantity.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productLevelDailyQuantity: org.apache.spark.sql.DataFrame = [productid: string, date: date ... 1 more field]
root
 |-- productid: string (nullable = true)
 |-- date: date (nullable = true)
 |-- quantity: double (nullable = true)



In [7]:
// var productLevelQuantity = (
//     productLevelDailyQuantity.filter(col("date")>=trainStartDate && col("date")<dateForWhichPredictionsMade).filter(!(col("date")===rundate))
//     .transform(getMonths("yyyy-MM-dd"))
//     .withColumnRenamed("monthsInHis", "pastmonth")
//     .filter(col("pastmonth") < 12)
//     .groupBy(PRODUCTID)
//     .agg(sum(QUANTITY) as QUANTITY)
//     .transform(saveNLoadDF("productQuantityForPast15MonthsAggregated"))
// )

var productLevelQuantity = load("productQuantityForPast15MonthsAggregated")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productLevelQuantity: org.apache.spark.sql.DataFrame = [productid: string, quantity: double]


In [8]:
// var ga = (
//     ParquetToDF.getDF("/data/ecomm/ajio/processed/ProcessedGAData")
//     .select(PRODUCTID, "PLPViewsPerDay", DATE)
//     .filter(col("date")>=trainStartDate && col("date")<dateForWhichPredictionsMade).filter(!(col("date")===rundate))
//     .transform(getMonths("yyyy-MM-dd"))
//     .withColumnRenamed("monthsInHis", "pastmonth")
//     .filter(col("pastmonth") < 12)
//     .groupBy(PRODUCTID)
//     .agg(sum("PLPViewsPerDay") as "impressions")
//     .transform(saveNLoadDF("productImpressionsForPast15MonthsAggregated"))
// )

var ga = load("productImpressionsForPast15MonthsAggregated")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ga: org.apache.spark.sql.DataFrame = [productid: string, impressions: double]


In [6]:
var productDetails = ParquetToDF.getDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/productDetails")
productDetails.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productDetails: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 9 more fields]
root
 |-- productid: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- colorfamily: string (nullable = true)
 |-- brandname: string (nullable = true)
 |-- styletype: string (nullable = true)
 |-- pattern: string (nullable = true)
 |-- sleeve: string (nullable = true)
 |-- price: double (nullable = true)
 |-- title: string (nullable = true)
 |-- imgcode: string (nullable = true)
 |-- category_name: string (nullable = true)



In [7]:
productDetails = productDetails.select("productid").distinct().transform(saveNLoadDF("allProducts"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productDetails: org.apache.spark.sql.DataFrame = [productid: string]


In [9]:
var productDetails = load("allProducts")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productDetails: org.apache.spark.sql.DataFrame = [productid: string]


In [10]:
productDetails.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res28: Long = 434089


In [12]:
var productMetrics = productDetails.join(productLevelQuantity, Seq(PRODUCTID), "left").join(ga, Seq(PRODUCTID), "left").na.fill(0).transform(saveNLoadDF("productLevelMetrics"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productMetrics: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [productid: string, quantity: double ... 1 more field]


In [11]:
var productMetrics = load("productLevelMetrics")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productMetrics: org.apache.spark.sql.DataFrame = [productid: string, quantity: double ... 1 more field]


In [12]:
var suffix = "colorfamily_pricebucket_styletype_pattern_sleeve_brandname"
var queriesToProductMap = ParquetToDF.getDF(s"$runDir/QueryProductMapping/suffix=$suffix").select("query", "productid", "similargrouplevel", "template").na.drop()
var oldQueryToNewQueryMap = ParquetToDF.getDF("/data/Archive/inventory/2023-04-30/queryProductMap_colorfamily_pricebucket_styletype_pattern_sleeve_brandname/attributesQueryInfo/final").select("query","template","similargrouplevel","newQuery")

queriesToProductMap = queriesToProductMap.join(oldQueryToNewQueryMap, Seq("similargrouplevel", "query", "template")).drop("query").withColumnRenamed("newQuery", "query")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

suffix: String = colorfamily_pricebucket_styletype_pattern_sleeve_brandname
queriesToProductMap: org.apache.spark.sql.DataFrame = [query: string, productid: string ... 2 more fields]
oldQueryToNewQueryMap: org.apache.spark.sql.DataFrame = [query: string, template: string ... 2 more fields]
queriesToProductMap: org.apache.spark.sql.DataFrame = [similargrouplevel: int, template: string ... 2 more fields]


In [13]:
queriesToProductMap.filter(col("query") =!= "#_#_#_#_#_#").show(5, false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+--------+---------------+-------------------+
|similargrouplevel|template|productid      |query              |
+-----------------+--------+---------------+-------------------+
|830216013        |pattern |410315517_01c  |#_#_solid_#_#_#    |
|830216013        |pattern |461102288_blue |#_#_check_#_#_#    |
|830216013        |pattern |461132549_khaki|#_#_solid_#_#_#    |
|830216013        |pattern |461575742_green|#_#_check_#_#_#    |
|830216013        |pattern |464973861_white|#_#_geometric_#_#_#|
+-----------------+--------+---------------+-------------------+
only showing top 5 rows



In [17]:
var queryLevelMetrics = (
    productMetrics.join(queriesToProductMap, Seq(PRODUCTID))
    .groupBy("query", "similargrouplevel")
    .agg(sum(QUANTITY).cast(DoubleType) as QUANTITY, sum("impressions").cast(DoubleType) as "impressions")
    .transform(saveNLoadDF("queryLevelMetrics"))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

queryLevelMetrics: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [query: string, similargrouplevel: int ... 2 more fields]


In [14]:
var queryLevelMetrics = load("queryLevelMetrics")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

queryLevelMetrics: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: int ... 2 more fields]


In [15]:
queryLevelMetrics.filter(col("query") === "#_black_#_4_full-length sleeve_#").show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------------+-----------------+--------+-----------+
|query                           |similargrouplevel|quantity|impressions|
+--------------------------------+-----------------+--------+-----------+
|#_black_#_4_full-length sleeve_#|830216013        |12117.0 |3.4713124E7|
|#_black_#_4_full-length sleeve_#|830303011        |181.0   |1197605.0  |
+--------------------------------+-----------------+--------+-----------+



In [16]:
queryLevelMetrics.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res32: Long = 1480430


In [17]:
var predictions = ParquetToDF.getDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/predictions")
predictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.sql.DataFrame = [similargrouplevel: string, category_name: string ... 8 more fields]
root
 |-- similargrouplevel: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- query: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- actual: double (nullable = true)



In [18]:
predictions.filter(col("query") === "#_black_#_4_full-length sleeve_#").show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-------------+--------------------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------+
|similargrouplevel|category_name|query                           |m1                |m2               |m3                |m4               |m5                |m6                |actual|
+-----------------+-------------+--------------------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------+
|830303011        |Women-Kurtas |#_black_#_4_full-length sleeve_#|7.647328495979309 |8.373806715011597|7.294650614261627 |8.877344787120819|9.22247713804245  |8.630289733409882 |3.0   |
|830216013        |Men-Shirts   |#_black_#_4_full-length sleeve_#|1574.0018355846405|1544.768319606781|1692.9795962572098|1677.558549940586|1979.5642426609993|1753.6360110640526|1713.0|
+-----------------+-------------+--------------------------------+----

In [23]:
predictions.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res36: Long = 765221


In [19]:
var impressions = (
    predictions.select(col("query"), col("similargrouplevel"), col("m1").as("predicted"), col("actual"))
    .join(queryLevelMetrics, Seq("query", SIMILAR_GROUP_LEVEL), "left")
    .na.fill(0)
    .withColumn("impressions", when(col("impressions") === 0 || col("quantity") === 0, lit(null)).otherwise((col("predicted") * col("impressions"))/col("quantity").cast(DoubleType)))
    .transform(saveNLoadDF("completeQuantityImpressionsQueryLevel"))
    .select("query", "similargrouplevel", "impressions")
)

DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/dashboardMetrics/impressions", impressions)
impressions = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/dashboardMetrics/impressions")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

impressions: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: string ... 1 more field]
impressions: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: string ... 1 more field]


In [20]:
var impressions = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/dashboardMetrics/impressions")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

impressions: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: string ... 1 more field]


In [21]:
impressions.filter(col("query") === "#_black_#_4_full-length sleeve_#").show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------------+-----------------+-----------------+
|query                           |similargrouplevel|impressions      |
+--------------------------------+-----------------+-----------------+
|#_black_#_4_full-length sleeve_#|830216013        |4509244.936442786|
|#_black_#_4_full-length sleeve_#|830303011        |50599.33062667017|
+--------------------------------+-----------------+-----------------+



In [22]:
impressions.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res38: Long = 765221


In [40]:
var discountProductLevel = ParquetToDF.getDF("/data/Archive/inventory/2023-04-30/queryProductMap_colorfamily_pricebucket_styletype_pattern_sleeve_brandname/productDiscountDashboard")
discountProductLevel.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

discountProductLevel: org.apache.spark.sql.DataFrame = [productid: string, WeightedAvgDiscount: double]
root
 |-- productid: string (nullable = true)
 |-- WeightedAvgDiscount: double (nullable = true)



In [41]:
discountProductLevel = discountProductLevel.join(productDetails, Seq(PRODUCTID))
discountProductLevel.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

discountProductLevel: org.apache.spark.sql.DataFrame = [productid: string, WeightedAvgDiscount: double]
res48: Long = 434089


In [42]:
var discountQueryLevel = (
    discountProductLevel.join(queriesToProductMap, Seq(PRODUCTID))
    .groupBy("query", "similargrouplevel")
    .agg(avg("WeightedAvgDiscount").cast(DoubleType) as "discount")
    .transform(saveNLoadDF("queryLevelDiscount"))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

discountQueryLevel: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [query: string, similargrouplevel: int ... 1 more field]


In [43]:
discountQueryLevel.filter(col("query") === "#_black_#_4_full-length sleeve_#").show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------------+-----------------+------------------+
|query                           |similargrouplevel|discount          |
+--------------------------------+-----------------+------------------+
|#_black_#_4_full-length sleeve_#|830216013        |14.039917502521597|
|#_black_#_4_full-length sleeve_#|830303011        |11.759362207500645|
+--------------------------------+-----------------+------------------+



In [44]:
var discount = (
    predictions.select("query", "similargrouplevel")
    .join(discountQueryLevel, Seq("query", SIMILAR_GROUP_LEVEL), "left")
    .na.fill(0.0)
    .select("query", "similargrouplevel", "discount")
)

DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/dashboardMetrics/discount", discount)
discount = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/dashboardMetrics/discount")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

discount: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: string ... 1 more field]
discount: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: string ... 1 more field]


In [23]:
var discount = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/dashboardMetrics/discount")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

discount: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: string ... 1 more field]


In [24]:
discount.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res39: Long = 765221


In [25]:
discount.show(5, false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------+-----------------+------------------+
|query                             |similargrouplevel|discount          |
+----------------------------------+-----------------+------------------+
|#_#_#_2_#_2piece-sets             |830303011        |70.0              |
|#_#_#_3_3-4th sleeve_anarkali     |830303011        |26.78706999565042 |
|#_#_#_3_full-length sleeve_classic|830216013        |30.552704830847677|
|#_#_#_4_full-length sleeve_utility|830216013        |33.98458287687867 |
|#_#_#_6_full-length sleeve_flared |830303011        |17.283471512952453|
+----------------------------------+-----------------+------------------+
only showing top 5 rows



In [48]:
impressions.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res54: Long = 765221


In [26]:
discount.select("discount").describe().show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|summary|discount          |
+-------+------------------+
|count  |765221            |
|mean   |50.266337388129976|
|stddev |21.091183787796652|
|min    |0.0               |
|max    |95.02251125562782 |
+-------+------------------+



### derive product level views from predicted impressions for query

In [27]:
var productMetrics = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/temp/dashboard_metrics/productLevelMetrics")
productMetrics.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productMetrics: org.apache.spark.sql.DataFrame = [productid: string, quantity: double ... 1 more field]
root
 |-- productid: string (nullable = true)
 |-- quantity: double (nullable = true)
 |-- impressions: double (nullable = true)



In [28]:
var queryLevelData = load("completeQuantityImpressionsQueryLevel")
queryLevelData.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

queryLevelData: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: string ... 4 more fields]
root
 |-- query: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- predicted: double (nullable = true)
 |-- actual: double (nullable = true)
 |-- quantity: double (nullable = true)
 |-- impressions: double (nullable = true)



In [29]:
var queryMetrics = productMetrics.join(queriesToProductMap, Seq(PRODUCTID)).transform(saveNLoadDF("queryWiseImpressionsQuantity"))
var queryMetricsStats = queryMetrics.groupBy("query", "similargrouplevel").agg(sum("impressions") as "sum_impressions", sum("quantity") as "sum_quantity")

queryMetrics = (
    queryMetrics.join(queryMetricsStats, Seq("query", "similargrouplevel"))
    .withColumn("viewsFraction", col("impressions")/col("sum_impressions").cast(DoubleType))
    .withColumn("quantityFraction", col("quantity")/col("sum_quantity").cast(DoubleType))
    .select("productid", "query", "similargrouplevel", "viewsFraction", "quantityFraction")
    .transform(saveNLoadDF("queryWiseProductFraction"))
    .join(queryLevelData, Seq("query", "similargrouplevel"))
    .withColumn("impressions", col("impressions") * col("viewsFraction"))
    .withColumn("predicted", col("predicted") * col("quantityFraction"))
    .withColumn("actual", col("actual") * col("quantityFraction"))
    .transform(saveNLoadDF("featuresProductLevelFromQueryLevel"))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

queryMetrics: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [productid: string, quantity: double ... 4 more fields]
queryMetricsStats: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: int ... 2 more fields]
queryMetrics: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [query: string, similargrouplevel: int ... 7 more fields]


In [30]:
queryMetrics.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- query: string (nullable = true)
 |-- similargrouplevel: integer (nullable = true)
 |-- productid: string (nullable = true)
 |-- viewsFraction: double (nullable = true)
 |-- quantityFraction: double (nullable = true)
 |-- predicted: double (nullable = true)
 |-- actual: double (nullable = true)
 |-- quantity: double (nullable = true)
 |-- impressions: double (nullable = true)



In [31]:
queryMetrics.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res46: Long = 20592793


In [32]:
queryMetrics.na.drop.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res47: Long = 20592304


In [33]:
DFToParquet.putDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/productPredictions", queryMetrics)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
queryMetrics.select(PRODUCTID).distinct.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res49: Long = 434089


### Dashboard Metrics

In [35]:
var aggregatedImpressions = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/dashboardMetrics/impressions")
var aggregatedDiscount = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/dashboardMetrics/discount")

var dashboardMetricsAggregated = aggregatedImpressions.join(aggregatedDiscount, Seq("query", "similargrouplevel"))
println(dashboardMetricsAggregated.count)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

aggregatedImpressions: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: string ... 1 more field]
aggregatedDiscount: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: string ... 1 more field]
dashboardMetricsAggregated: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: string ... 2 more fields]
765221


In [36]:
var dashboardMetricsAvg = (
    load("featuresProductLevelFromQueryLevel").filter(col("impressions") > 0 && !col("impressions").isNull && col("quantityFraction") > 0)
    .groupBy("query", "similargrouplevel")
    .agg(
        struct(
            min("predicted").as("min"), 
            callUDF("percentile", col("predicted"), lit(0.25)).as("percentile_25"),
            callUDF("percentile", col("predicted"), lit(0.5)).as("percentile_50"),
            callUDF("percentile", col("predicted"), lit(0.75)).as("percentile_75"),
            max("predicted").as("max")
        ) as "predicted_stats",
        struct(
            min("impressions").as("min"), 
            callUDF("percentile", col("impressions"), lit(0.25)).as("percentile_25"),
            callUDF("percentile", col("impressions"), lit(0.5)).as("percentile_50"),
            callUDF("percentile", col("impressions"), lit(0.75)).as("percentile_75"),
            max("impressions").as("max")
        ) as "impressions_stats",
        struct(
            min("actual").as("min"), 
            callUDF("percentile", col("actual"), lit(0.25)).as("percentile_25"),
            callUDF("percentile", col("actual"), lit(0.5)).as("percentile_50"),
            callUDF("percentile", col("actual"), lit(0.75)).as("percentile_75"),
            max("actual").as("max")
        ) as "actual_stats",
        avg("impressions") as "avg_impressions",
        avg("predicted") as "avg_predicted",
        avg("actual") as "avg_actual",
        count("productid") as "numProducts"
    )
)
dashboardMetricsAvg.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dashboardMetricsAvg: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: int ... 7 more fields]
root
 |-- query: string (nullable = true)
 |-- similargrouplevel: integer (nullable = true)
 |-- predicted_stats: struct (nullable = false)
 |    |-- min: double (nullable = true)
 |    |-- percentile_25: double (nullable = true)
 |    |-- percentile_50: double (nullable = true)
 |    |-- percentile_75: double (nullable = true)
 |    |-- max: double (nullable = true)
 |-- impressions_stats: struct (nullable = false)
 |    |-- min: double (nullable = true)
 |    |-- percentile_25: double (nullable = true)
 |    |-- percentile_50: double (nullable = true)
 |    |-- percentile_75: double (nullable = true)
 |    |-- max: double (nullable = true)
 |-- actual_stats: struct (nullable = false)
 |    |-- min: double (nullable = true)
 |    |-- percentile_25: double (nullable = true)
 |    |-- percentile_50: double (nullable = true)
 |    |-- percentile_75: double (nullable = true)
 | 

In [37]:
dashboardMetricsAvg.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res53: Long = 762659


In [38]:
dashboardMetricsAvg.show(5, false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------+-----------------+------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------+------------------+------------------+-------------------+-----------+
|query                                          |similargrouplevel|predicted_stats                                                                                       |impressions_stats                                                                                  |actual_stats                                                                                              |avg_impressions   |avg_predicted     |avg_actual         |numProducts|
+-----------------------------------------------+-----------------+-------------------------------------

In [39]:
dashboardMetricsAggregated.show(5, false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------+-----------------+-------------------+------------------+
|query                             |similargrouplevel|impressions        |discount          |
+----------------------------------+-----------------+-------------------+------------------+
|#_#_#_2_#_2piece-sets             |830303011        |136.69046905636787 |70.0              |
|#_#_#_3_3-4th sleeve_anarkali     |830303011        |8775390.589225857  |26.78706999565042 |
|#_#_#_3_full-length sleeve_classic|830216013        |2.038563718529041E8|30.552704830847677|
|#_#_#_4_full-length sleeve_utility|830216013        |260325.6715233877  |33.98458287687867 |
|#_#_#_6_full-length sleeve_flared |830303011        |61640.09917333421  |17.283471512952453|
+----------------------------------+-----------------+-------------------+------------------+
only showing top 5 rows



In [40]:
dashboardMetricsAggregated.filter(col("query") === "#_#_#_2_short sleeve_indian").show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------+-----------------+-----------------+-----------------+
|query                      |similargrouplevel|impressions      |discount         |
+---------------------------+-----------------+-----------------+-----------------+
|#_#_#_2_short sleeve_indian|830216013        |5279109.332190486|41.53878442868594|
+---------------------------+-----------------+-----------------+-----------------+



In [79]:
dashboardMetricsAggregated.filter(col("query") === "#_#_#_2_short sleeve_indian").show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------+-----------------+-----------------+-----------------+
|query                      |similargrouplevel|impressions      |discount         |
+---------------------------+-----------------+-----------------+-----------------+
|#_#_#_2_short sleeve_indian|830216013        |5279109.332190486|41.53878442868594|
+---------------------------+-----------------+-----------------+-----------------+



In [41]:
var dashboardMetrics = dashboardMetricsAggregated.join(dashboardMetricsAvg, Seq("query", "similargrouplevel"), "left")
dashboardMetrics.printSchema
println(dashboardMetrics.count)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dashboardMetrics: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: string ... 9 more fields]
root
 |-- query: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- impressions: double (nullable = true)
 |-- discount: double (nullable = true)
 |-- predicted_stats: struct (nullable = true)
 |    |-- min: double (nullable = true)
 |    |-- percentile_25: double (nullable = true)
 |    |-- percentile_50: double (nullable = true)
 |    |-- percentile_75: double (nullable = true)
 |    |-- max: double (nullable = true)
 |-- impressions_stats: struct (nullable = true)
 |    |-- min: double (nullable = true)
 |    |-- percentile_25: double (nullable = true)
 |    |-- percentile_50: double (nullable = true)
 |    |-- percentile_75: double (nullable = true)
 |    |-- max: double (nullable = true)
 |-- actual_stats: struct (nullable = true)
 |    |-- min: double (nullable = true)
 |    |-- percentile_25: double (nullable = true)
 |    |-- percentile_50:

In [42]:
DFToParquet.putDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/dashboardMetrics", dashboardMetrics)
dashboardMetrics = ParquetToDF.getDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/dashboardMetrics")

dashboardMetrics.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dashboardMetrics: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: string ... 9 more fields]
root
 |-- query: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- impressions: double (nullable = true)
 |-- discount: double (nullable = true)
 |-- predicted_stats: struct (nullable = true)
 |    |-- min: double (nullable = true)
 |    |-- percentile_25: double (nullable = true)
 |    |-- percentile_50: double (nullable = true)
 |    |-- percentile_75: double (nullable = true)
 |    |-- max: double (nullable = true)
 |-- impressions_stats: struct (nullable = true)
 |    |-- min: double (nullable = true)
 |    |-- percentile_25: double (nullable = true)
 |    |-- percentile_50: double (nullable = true)
 |    |-- percentile_75: double (nullable = true)
 |    |-- max: double (nullable = true)
 |-- actual_stats: struct (nullable = true)
 |    |-- min: double (nullable = true)
 |    |-- percentile_25: double (nullable = true)
 |    |-- percentile_50:

In [43]:
dashboardMetrics.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res62: Long = 765221


In [44]:
dashboardMetrics.filter(col("query") === "#_#_#_#_#_#").count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res63: Long = 0


In [45]:
dashboardMetrics.filter(col("query") === "#_beige_#_1_3-4th sleeve_a-line").show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------+-----------------+-----------------+-----------------+-------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------+------------------+------------------+------------------+-----------+
|query                          |similargrouplevel|impressions      |discount         |predicted_stats                                                                                  |impressions_stats                                                                                |actual_stats                                                                                         |avg_impressions   |avg_predicted     |avg_actual        |numProducts|
+-------------------------------+-----------------+-----------------+-----------------+---

In [46]:
dashboardMetrics.filter(col("predicted_stats").isNull).show(1, false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------+-----------------+-----------+-----------------+---------------+-----------------+------------+---------------+-------------+----------+-----------+
|query                        |similargrouplevel|impressions|discount         |predicted_stats|impressions_stats|actual_stats|avg_impressions|avg_predicted|avg_actual|numProducts|
+-----------------------------+-----------------+-----------+-----------------+---------------+-----------------+------------+---------------+-------------+----------+-----------+
|abhishti_#_solid_4_#_anarkali|830303011        |0.0        |38.77367939461646|null           |null             |null        |null           |null         |null      |null       |
+-----------------------------+-----------------+-----------+-----------------+---------------+-----------------+------------+---------------+-------------+----------+-----------+
only showing top 1 row



In [83]:
dashboardMetrics.filter(col("query") === "#_beige_#_1_3-4th sleeve_a-line").show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------+-----------------+-----------------+-----------------+-------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------+------------------+-----------------+-----------+
|query                          |similargrouplevel|impressions      |discount         |predicted_stats                                                                                  |impressions_stats                                                                                |avg_impressions   |avg_predicted    |numProducts|
+-------------------------------+-----------------+-----------------+-----------------+-------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------+------------------+-----------------+-----------+
|

In [84]:
dashboardMetrics.filter(col("predicted_stats").isNull).show(1, false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------+-----------------+-----------+-----------------+---------------+-----------------+---------------+-------------+-----------+
|query                        |similargrouplevel|impressions|discount         |predicted_stats|impressions_stats|avg_impressions|avg_predicted|numProducts|
+-----------------------------+-----------------+-----------+-----------------+---------------+-----------------+---------------+-------------+-----------+
|abhishti_#_solid_4_#_anarkali|830303011        |0.0        |38.77367939461646|null           |null             |null           |null         |null       |
+-----------------------------+-----------------+-----------+-----------------+---------------+-----------------+---------------+-------------+-----------+
only showing top 1 row



### New

In [86]:
var productPredictions = ParquetToDF.getDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/productPredictions")
productPredictions.printSchema

var predictions = ParquetToDF.getDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/predictions")
predictions.printSchema

var dashboardMetrics = ParquetToDF.getDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/dashboardMetrics")
dashboardMetrics.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productPredictions: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: int ... 6 more fields]
root
 |-- query: string (nullable = true)
 |-- similargrouplevel: integer (nullable = true)
 |-- productid: string (nullable = true)
 |-- viewsFraction: double (nullable = true)
 |-- quantityFraction: double (nullable = true)
 |-- predicted: double (nullable = true)
 |-- quantity: double (nullable = true)
 |-- impressions: double (nullable = true)

predictions: org.apache.spark.sql.DataFrame = [similargrouplevel: string, category_name: string ... 7 more fields]
root
 |-- similargrouplevel: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- query: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)

dashboardMetrics: org.apache.spark.sql.DataFrame = [query: string, similargrouplev

In [87]:
predictions.filter(col("query") === "campus-sutra_multi_stripes_2_short sleeve_classic").show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-------------+-------------------------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|similargrouplevel|category_name|query                                            |m1                |m2                |m3                |m4                |m5                |m6                |
+-----------------+-------------+-------------------------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|830216013        |Men-Shirts   |campus-sutra_multi_stripes_2_short sleeve_classic|1041.2389748096466|1027.6615380048752|1007.5732480287552|1135.5232311487198|1084.8571891784668|1156.5783071517944|
+-----------------+-------------+-------------------------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+



In [88]:
dashboardMetrics.filter(col("query") === "campus-sutra_multi_stripes_2_short sleeve_classic").show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------------+-----------------+------------------+-----------------+---------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------+----------------+-----------------+-----------+
|query                                            |similargrouplevel|impressions       |discount         |predicted_stats                                                                                    |impressions_stats                                                                                |avg_impressions |avg_predicted    |numProducts|
+-------------------------------------------------+-----------------+------------------+-----------------+---------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------

In [89]:
productPredictions.filter(col("query") === "campus-sutra_multi_stripes_2_short sleeve_classic").agg(sum("predicted"), sum("impressions")).show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+
|sum(predicted)    |sum(impressions)  |
+------------------+------------------+
|1041.2389748096466|1477749.1999075578|
+------------------+------------------+



#### Deprecated (query level hitrate) as not calculating actual at product level anymore

In [22]:
var productLevelSigma = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/Statistics12Months/product").select("productid", "stddev")
productLevelSigma.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productLevelSigma: org.apache.spark.sql.DataFrame = [productid: string, stddev: double]
root
 |-- productid: string (nullable = true)
 |-- stddev: double (nullable = true)



In [32]:
var productPreds = productPredictions.filter(col("impressions").isNotNull && col("impressions")>0 && col("quantityFraction")>0).select("productid", "query", "predicted", "similargrouplevel", "actual", "template")
productPreds.printSchema
println(productPreds.count)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productPreds: org.apache.spark.sql.DataFrame = [productid: string, query: string ... 4 more fields]
root
 |-- productid: string (nullable = true)
 |-- query: string (nullable = true)
 |-- predicted: double (nullable = true)
 |-- similargrouplevel: integer (nullable = true)
 |-- actual: double (nullable = true)
 |-- template: string (nullable = true)

7978258


In [33]:
productPreds.join(productLevelSigma, Seq(PRODUCTID)).count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res43: Long = 7978258


In [34]:
def thresholdColumns(columns: Array[String], minThreshold: Double = 0, maxThreshold: Double = Double.PositiveInfinity)(df: DataFrame): DataFrame = {

    var scaledDF = df
    columns.foreach(
      column => {
        scaledDF = scaledDF.withColumn(column,
          when(col(column) > maxThreshold, lit(maxThreshold)).otherwise(col(column)))
          .withColumn(column, when(col(column) < minThreshold, lit(minThreshold)).otherwise(col(column)))
      })
    scaledDF
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

thresholdColumns: (columns: Array[String], minThreshold: Double, maxThreshold: Double)(df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


In [35]:
productPreds = productPreds.join(productLevelSigma, Seq(PRODUCTID))
productPreds = (
    productPreds.withColumn("lowerBound", col("predicted") - (lit(2.0) * col("stddev")))
      .withColumn("upperBound", col("predicted") + (lit(2.0) * col("stddev")))
      .transform(thresholdColumns(Array("lowerBound")))
)
productPreds.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productPreds: org.apache.spark.sql.DataFrame = [productid: string, query: string ... 5 more fields]
productPreds: org.apache.spark.sql.DataFrame = [productid: string, query: string ... 7 more fields]
root
 |-- productid: string (nullable = true)
 |-- query: string (nullable = true)
 |-- predicted: double (nullable = true)
 |-- similargrouplevel: integer (nullable = true)
 |-- actual: double (nullable = true)
 |-- template: string (nullable = true)
 |-- stddev: double (nullable = true)
 |-- lowerBound: double (nullable = true)
 |-- upperBound: double (nullable = true)



In [40]:
productPreds = productPreds.withColumn("isHit", (col("actual") >= col("lowerBound") && col("actual") <= col("upperBound")).cast(DoubleType))
productPreds.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productPreds: org.apache.spark.sql.DataFrame = [productid: string, query: string ... 8 more fields]
root
 |-- productid: string (nullable = true)
 |-- query: string (nullable = true)
 |-- predicted: double (nullable = true)
 |-- similargrouplevel: integer (nullable = true)
 |-- actual: double (nullable = true)
 |-- template: string (nullable = true)
 |-- stddev: double (nullable = true)
 |-- lowerBound: double (nullable = true)
 |-- upperBound: double (nullable = true)
 |-- isHit: double (nullable = true)



In [41]:
var hitRateDF = productPreds.groupBy("query", "similargrouplevel", "template").agg(sum("isHit")/count("productid") as "hitRate")
hitRateDF.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hitRateDF: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: int ... 2 more fields]
root
 |-- query: string (nullable = true)
 |-- similargrouplevel: integer (nullable = true)
 |-- template: string (nullable = true)
 |-- hitRate: double (nullable = true)



In [42]:
hitRateDF.show(5, false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------+-----------------+-------------------------------------------+------------------+
|query                              |similargrouplevel|template                                   |hitRate           |
+-----------------------------------+-----------------+-------------------------------------------+------------------+
|enchanted-drapes_blue_#_1_#_classic|830216013        |pricebucket_colorfamily_styletype_brandname|0.6666666666666666|
|#_lavender_embroidery_#_#_#        |830303011        |pattern_colorfamily                        |0.4166666666666667|
|showoff_#_floral_6_#_anarkali      |830303011        |pattern_pricebucket_styletype_brandname    |1.0               |
|sringam_green_floral_#_#_#         |830303011        |pattern_colorfamily_brandname              |0.5625            |
|rangmayee_red_floral_#_#_flared    |830303011        |pattern_colorfamily_styletype_brandname    |0.0               |
+-----------------------------------+-----------

In [43]:
hitRateDF.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res51: Long = 383183


In [44]:
var predictions = ParquetToDF.getDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/predictions").select("query", "similargrouplevel", "template").distinct
predictions.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [query: string, similargrouplevel: int ... 1 more field]
res52: Long = 383205


In [45]:
hitRateDF = predictions.join(hitRateDF, Seq("query", "similargrouplevel", "template"), "left")
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/dashboardMetrics/hitRate", hitRateDF)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hitRateDF: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: int ... 2 more fields]
